<a href="https://colab.research.google.com/github/Pathway2008/wind-m-s-/blob/main/blending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/wind(m s)/train.csv')
test = pd.read_csv('/content/drive/MyDrive/wind(m s)/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/wind(m s)/sample_submission.csv')

In [2]:
def map_wind_direction(deg):
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    angles = [(0 or 360), 22.5, 45, 67.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5]

    # Find the closest wind direction from the given degrees
    closest_idx = min(range(len(angles)), key=lambda i: abs(angles[i] - deg))
    return directions[closest_idx]

In [3]:
train['방위'] =train['풍향 (deg)'].apply(map_wind_direction)
test['방위'] =test['풍향 (deg)'].apply(map_wind_direction)

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
ordinal_features = ['측정 시간대','방위']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [6]:
train.head()

,ID,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s),방위
0,TRAIN_00000,7,2,3,13.97,287.78,9.84,76.1,992.08,15.98,12.16,3.82,7.66,1198.06,155.60,1.61,10
1,TRAIN_00001,8,21,1,16.94,290.85,12.14,73.3,991.07,19.33,14.17,5.16,8.94,1183.67,177.00,1.68,8
2,TRAIN_00002,11,1,3,9.76,283.84,5.40,74.2,988.71,12.10,8.98,3.12,5.67,1213.22,146.20,0.73,9
3,TRAIN_00003,12,28,1,5.27,277.30,2.71,83.5,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50,2.71,13
4,TRAIN_00004,9,26,2,17.35,290.86,12.68,74.0,995.77,19.84,14.68,5.16,9.22,1187.40,19.34,1.00,5


In [7]:
test.head()

,ID,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),방위
0,TEST_00000,3,24,0,4.28,278.68,-0.21,72.5,984.48,8.30,6.02,2.28,3.81,1233.29,251.80,15
1,TEST_00001,9,24,3,13.40,286.81,10.36,81.8,996.98,15.40,12.59,2.80,7.89,1206.20,225.60,12
2,TEST_00002,5,28,3,19.89,294.33,14.95,73.2,984.83,23.26,17.03,6.23,10.82,1163.06,10.39,5
3,TEST_00003,1,17,3,-2.88,270.44,-4.47,88.7,998.02,4.94,4.38,0.56,2.73,1284.19,260.20,13
4,TEST_00004,10,22,2,6.97,281.18,4.36,83.4,987.00,10.01,8.35,1.66,5.28,1223.47,262.50,13


In [6]:
st_train  = train.drop(['ID'],axis=1)

In [7]:
columns = st_train.drop(columns=['측정 시간대','월', '일', '풍속 (m/s)','방위']).columns
columns

Index(['섭씨 온도(°⁣C)', '절대 온도(K)', '이슬점 온도(°C)', '상대 습도 (%)', '대기압(mbar)',
       '포화 증기압(mbar)', '실제 증기압(mbar)', '증기압 부족량(mbar)', '수증기 함량 (g/kg)',
       '공기 밀도 (g/m**3)', '풍향 (deg)'],
      dtype='object')

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_scaled = scaler.fit_transform(st_train[columns])
st_train[columns] = train_scaled
st_train.head()

,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s),방위
0,7,2,3,0.482855,0.455931,0.693376,0.016942,0.231977,0.240582,0.566963,-0.082878,0.558184,-0.417949,-0.248661,1.61,10
1,8,21,1,0.843420,0.823415,1.056902,-0.157090,0.117521,0.655413,1.047176,0.175258,1.040291,-0.781995,0.004949,1.68,8
2,11,1,3,-0.028250,-0.015692,-0.008387,-0.101151,-0.149921,-0.239880,-0.192777,-0.217725,-0.191340,-0.034423,-0.360060,0.73,9
3,12,28,1,-0.573347,-0.798540,-0.433554,0.476885,2.744341,-0.637375,-0.563090,-0.535580,-0.605650,1.287678,1.041906,2.71,13
4,9,26,2,0.893195,0.824612,1.142251,-0.113582,0.650138,0.718566,1.169021,0.175258,1.145751,-0.687631,-1.863469,1.00,5


In [9]:
st_test = test.drop(['ID'],axis=1)
test_scaled = scaler.transform(test[columns])
st_test[columns] = test_scaled
st_test.head()

,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),방위
0,3,24,0,-0.693535,-0.633352,-0.895074,-0.206814,-0.629276,-0.710435,-0.899956,-0.379542,-0.891900,0.473318,0.891399,15
1,9,24,3,0.413655,0.339821,0.775564,0.371222,0.787258,0.168760,0.669695,-0.279370,0.644813,-0.212019,0.580904,12
2,5,28,3,1.201557,1.239975,1.501036,-0.163306,-0.589613,1.142066,1.730464,0.381382,1.748384,-1.303398,-1.969535,5
3,1,17,3,-1.562777,-1.619691,-1.568387,0.800087,0.905114,-1.126505,-1.291771,-0.710881,-1.298677,1.761014,0.990947,13
4,10,22,2,-0.366963,-0.334098,-0.172764,0.470669,-0.343702,-0.498685,-0.343291,-0.498978,-0.338231,0.224887,1.018204,13


In [10]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 114.7 MB/s eta 0:0

In [17]:
import pandas as pd
from pycaret.regression import setup, compare_models, finalize_model, predict_model,create_model
from sklearn.metrics import mean_absolute_error

In [12]:
target = '풍속 (m/s)'
features = st_train.columns.drop(['풍속 (m/s)'])
regression_setup = setup(data=st_train, target=target, train_size=0.8, session_id=42)

,Description,Value
0,Session id,42
1,Target,풍속 (m/s)
2,Target type,Regression
3,Original data shape,"(36581, 16)"
4,Transformed data shape,"(36581, 16)"
5,Transformed train set shape,"(29264, 16)"
6,Transformed test set shape,"(7317, 16)"
7,Numeric features,15
8,Preprocess,True
9,Imputation type,simple


In [13]:
best_model = compare_models(fold=5, sort='MAE', n_select=4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.4213,0.3318,0.5759,0.8590,0.1974,0.3595,8.1440
rf,Random Forest Regressor,0.4679,0.4126,0.6423,0.8247,0.2195,0.4126,23.6400
knn,K Neighbors Regressor,0.5042,0.4933,0.7023,0.7904,0.2341,0.4267,0.2260
xgboost,Extreme Gradient Boosting,0.5521,0.5500,0.7416,0.7663,0.2486,0.4592,3.8900
lightgbm,Light Gradient Boosting Machine,0.6018,0.6363,0.7977,0.7296,0.2657,0.5204,0.9200
dt,Decision Tree Regressor,0.6153,0.8212,0.9060,0.6509,0.2979,0.4915,0.4140
gbr,Gradient Boosting Regressor,0.7734,1.0497,1.0245,0.5540,0.3254,0.6667,6.8140
huber,Huber Regressor,0.9871,1.9762,1.4058,0.1603,0.4016,0.7625,0.9720
br,Bayesian Ridge,1.0058,1.8303,1.3529,0.2222,0.4081,0.8707,0.2380
lr,Linear Regression,1.0059,1.8303,1.3529,0.2222,0.4082,0.8707,1.7000


Processing:   0%|          | 0/84 [00:00<?, ?it/s]

In [14]:
final_model = finalize_model(best_model[0])
predictions = predict_model(final_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000


In [35]:
et_params = best_model[0].get_params()
rf_params = best_model[1].get_params()
#kng_params = best_model[2].get_params()
xgb_params = best_model[3].get_params()

In [36]:
kng_params = {'n_neighbors': 4, 'weights': 'distance', 'p': 2, 'leaf_size': 41, 'algorithm': 'brute'}

In [37]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [21]:
X = st_train.drop(['풍속 (m/s)'], axis=1)
y = st_train['풍속 (m/s)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
et = ExtraTreesRegressor(**et_params)
et.fit(X_train, y_train)
et_pred = et.predict(X_test)
extra_trees_mae = mean_absolute_error(y_test, et_pred)
print("Extra Trees Regressor MAE:", extra_trees_mae)

rf = RandomForestRegressor(**rf_params)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
random_forest_mae = mean_absolute_error(y_test, rf_pred)
print("Random Forest Regressor MAE:", random_forest_mae)


knr = KNeighborsRegressor(**kng_params)
knr.fit(X_train, y_train)
knr_pred = knr.predict(X_test)
k_neighbors_mae = mean_absolute_error(y_test, knr_pred)
print("K Neighbors Regressor MAE:", k_neighbors_mae)

xgb= xgb.XGBRegressor(**xgb_params)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgboost_mae = mean_absolute_error(y_test, xgb_pred)
print("XGBoost Regressor MAE:", xgboost_mae)

Extra Trees Regressor MAE: 0.4034034167008337
Random Forest Regressor MAE: 0.45255474921415884
K Neighbors Regressor MAE: 0.437127741307292
XGBoost Regressor MAE: 0.544517623302306


In [39]:
import numpy as np

# Define the number of points for each weight
num_points = 10

# Generate continuous weights between 0 and 1 (inclusive) with 'num_points' points
weights_range = np.linspace(0, 1, num_points)

# Create all possible combinations of weights using NumPy's meshgrid function
w1, w2, w3, w4 = np.meshgrid(weights_range, weights_range, weights_range, weights_range)
possible_weights_continuous = np.stack((w1, w2, w3, w4), axis=-1)

# Filter out combinations where the sum is not equal to 1 (within a small tolerance)
tolerance = 1e-6
possible_weights_continuous = possible_weights_continuous[
    np.abs(np.sum(possible_weights_continuous, axis=-1) - 1) < tolerance
]

print("Continuous Possible Weights:")
print(possible_weights_continuous)

Continuous Possible Weights:
[[0.         0.         0.         1.        ]
 [0.         0.         0.11111111 0.88888889]
 [0.         0.         0.22222222 0.77777778]
 [0.         0.         0.33333333 0.66666667]
 [0.         0.         0.44444444 0.55555556]
 [0.         0.         0.55555556 0.44444444]
 [0.         0.         0.66666667 0.33333333]
 [0.         0.         0.77777778 0.22222222]
 [0.         0.         0.88888889 0.11111111]
 [0.         0.         1.         0.        ]
 [0.11111111 0.         0.         0.88888889]
 [0.11111111 0.         0.11111111 0.77777778]
 [0.11111111 0.         0.22222222 0.66666667]
 [0.11111111 0.         0.33333333 0.55555556]
 [0.11111111 0.         0.44444444 0.44444444]
 [0.11111111 0.         0.55555556 0.33333333]
 [0.11111111 0.         0.66666667 0.22222222]
 [0.11111111 0.         0.77777778 0.11111111]
 [0.11111111 0.         0.88888889 0.        ]
 [0.22222222 0.         0.         0.77777778]
 [0.22222222 0.         0.11111

In [25]:
from sklearn.model_selection import GridSearchCV

In [29]:
def blend_predictions(weights, predictions):
    # Ensure the weights are normalized to sum up to 1
    weights = np.array(weights) / np.sum(weights)

    # Element-wise multiplication between each model's prediction and its corresponding weight
    weighted_predictions = [weight * pred for weight, pred in zip(weights, predictions)]

    # Sum up the weighted predictions along the first axis (axis=0) to get the final blended prediction
    blended_prediction = np.sum(weighted_predictions, axis=0)

    return blended_prediction

In [30]:
possible_weights = [
    [w1, w2, w3, w4] for w1 in [0.1, 0.3, 0.5, 0.7, 0.9]
    for w2 in [0.1, 0.3, 0.5, 0.7, 0.9]
    for w3 in [0.1, 0.3, 0.5, 0.7, 0.9]
    for w4 in [0.1, 0.3, 0.5, 0.7, 0.9]
    if w1 + w2 + w3 + w4 == 1
]

best_weights = None
best_mae = float('inf')

In [31]:
for weights in possible_weights:
    blended_pred = blend_predictions(weights, [et_pred, rf_pred, knr_pred, xgb_pred])
    blended_mae = mean_absolute_error(y_test, blended_pred)

    if blended_mae < best_mae:
        best_mae = blended_mae
        best_weights = weights

In [32]:
print("Best Blending Weights:", best_weights)
print("Best Blended Model MAE:", best_mae)

Best Blending Weights: [0.5, 0.3, 0.1, 0.1]
Best Blended Model MAE: 0.4174338632114975
